In [1]:
'''
    Opening notebook remotely on MacOS --> jupyter-notebook --no-browser --port=9090
'''

'\n    Opening notebook remotely on MacOS --> jupyter-notebook --no-browser --port=9090\n'

In [2]:
from sklearn.metrics import classification_report
from keras.models import load_model
import numpy as np
from sklearn.metrics import roc_curve, auc
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-11-18 11:37:04.234033: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 11:37:04.236536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/jvm/java-11-openjdk/lib
2023-11-18 11:37:04.236545: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
shared_dir = "/s/bach/a/class/cs435/cs435a/"
image_dir = "CS435_Plant_Data_v2_augment-v2/2023-11-15-231056/"

orig_test = "splits/original/orig_test.csv"
aug_test = "splits/augmented/aug_test.csv"

orig_test_path = shared_dir+image_dir+orig_test
aug_test_path = shared_dir+image_dir+aug_test

In [5]:
orig_model_name = shared_dir + "models/trained_original.h5"
aug_model_name = shared_dir + "models/trained_augmented.h5"
orig_model = load_model(orig_model_name)
aug_model = load_model(aug_model_name)

In [6]:
orig_test_df = pd.read_csv(orig_test_path)
aug_test_df = pd.read_csv(aug_test_path)

datagen = ImageDataGenerator(rescale=1./255) #normalize from [0,255] to [0,1]
orig_test_generator = datagen.flow_from_dataframe(
    dataframe=orig_test_df, 
    x_col='path_to_shared',
    y_col='class_name',  
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical', 
    shuffle=False
)

aug_test_generator = datagen.flow_from_dataframe(
    dataframe=aug_test_df, 
    x_col='path_to_shared',
    y_col='class_name',  
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical', 
    shuffle=False
)

Found 7166 validated image filenames belonging to 31 classes.
Found 21580 validated image filenames belonging to 31 classes.


In [11]:
# Basic metrics: accuracy, precision, recall, f1-score
def evaluate_model_metrics(model, test_generator):
    predictions = model.predict(test_generator)
    predicted_classes = [np.argmax(pred) for pred in predictions]

    true_classes = test_generator.classes

    class_labels = list(test_generator.class_indices.keys())

    report = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)
    return report

In [26]:
# More advanced metrics: Confusion Matrix
def create_model_confusion_matrix(model, test_generator, model_name):
    predictions = model.predict(test_generator)
    
    predicted_classes = np.argmax(predictions, axis=1)

    label_map = {label: index for index, label in enumerate(test_generator.class_indices)}

    true_classes = test_df['class_name'].map(label_map).values

    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    plt.figure(figsize=(20, 20))
    sns.heatmap(conf_matrix, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')

    # plt.savefig(model_name+'_aug_confusion_matrix.png')
    plt.show()

In [12]:
print("*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***")
orig_orig_test_report = evaluate_model_metrics(orig_model, orig_test_generator)
orig_aug_test_report = evaluate_model_metrics(orig_model, aug_test_generator)

*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***
1792/1792 [==============================] - 358s 200ms/step - loss: 0.2655 - accuracy: 0.9112
Test Accuracy: 0.9112475514411926
21105/21580 [============================>.] - ETA: 30s - loss: 1.5024 - accuracy: 0.6507

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



21580/21580 [==============================] - 1378s 64ms/step
*** EVALUATIONS ON ORIGINAL TEST DATA ***


AttributeError: module 'pandas' has no attribute 'Dataframe'

In [25]:
'''
    TODO: GET AVERAGE RIGHT BASED ON MACRO AVG NOT AVG OF THE COLUMN!
'''

print("*** EVALUATIONS ON ORIGINAL TEST DATA WITH MODEL TRAINED ON ORIGINAL DATA ***")
orig_orig_df = pd.DataFrame(orig_orig_test_report).transpose()

print(f"Accuracy: {orig_orig_df.loc['accuracy', 'precision']}")
print(f"Macro Average Precision: {aug_orig_df.loc['macro avg', 'precision']}")
print(f"Macro Average Recall: {aug_orig_df.loc['macro avg', 'recall']}")
print(f"Macro Average F1-Score: {aug_orig_df.loc['macro avg', 'f1-score']}")

display(orig_orig_df)

print("*** EVALUATIONS ON AUGMENTED TEST DATA WITH MODEL TRAINED ON ORIGINAL DATA ***")
orig_aug_df = pd.DataFrame(orig_aug_test_report).transpose()

print(f"Accuracy: {orig_aug_df.loc['accuracy', 'precision']}")
print(f"Macro Average Precision: {orig_aug_df.loc['macro avg', 'precision']}")
print(f"Macro Average Recall: {orig_aug_df.loc['macro avg', 'recall']}")
print(f"Macro Average F1-Score: {orig_aug_df.loc['macro avg', 'f1-score']}")

display(orig_aug_df)


*** EVALUATIONS ON ORIGINAL TEST DATA WITH MODEL TRAINED ON ORIGINAL DATA ***
Accuracy: 0.9112475579123639
Macro Average Precision: 0.9392576863536143
Macro Average Recall: 0.9319632752592761
Macro Average F1-Score: 0.9305437187308214


,precision,recall,f1-score,support
Apple___Apple_scab,0.985915,0.886076,0.933333,237.000000
Apple___Black_rot,0.995868,0.902622,0.946955,267.000000
Apple___Cedar_apple_rust,0.962185,0.987069,0.974468,232.000000
Apple___healthy,0.820261,0.984314,0.894831,255.000000
Cherry_(including_sour)___Powdery_mildew,0.994681,0.973958,0.984211,192.000000
Cherry_(including_sour)___healthy,0.995215,0.967442,0.981132,215.000000
Corn_(maize)___Common_rust_,1.000000,0.990826,0.995392,218.000000
Corn_(maize)___Northern_Leaf_Blight,0.985366,0.926606,0.955083,218.000000
Corn_(maize)___healthy,0.936330,1.000000,0.967118,250.000000
Grape___Black_rot,0.943089,0.935484,0.939271,248.000000


*** EVALUATIONS ON AUGMENTED TEST DATA WITH MODEL TRAINED ON ORIGINAL DATA ***
Accuracy: 0.6505560704355885
Macro Average Precision: 0.8098064551064673
Macro Average Recall: 0.6504238669469813
Macro Average F1-Score: 0.670082182436296


,precision,recall,f1-score,support
Apple___Apple_scab,0.962871,0.514550,0.670690,756.000000
Apple___Black_rot,0.793556,0.892617,0.840177,745.000000
Apple___Cedar_apple_rust,0.556008,0.827273,0.665043,660.000000
Apple___healthy,0.805687,0.677291,0.735931,753.000000
Cherry_(including_sour)___Powdery_mildew,0.963899,0.423138,0.588106,631.000000
Cherry_(including_sour)___healthy,0.991561,0.686131,0.811044,685.000000
Corn_(maize)___Common_rust_,0.754860,0.977622,0.851920,715.000000
Corn_(maize)___Northern_Leaf_Blight,0.767878,0.916084,0.835459,715.000000
Corn_(maize)___healthy,0.888889,0.677188,0.768730,697.000000
Grape___Black_rot,0.708565,0.899718,0.792782,708.000000


In [19]:
print("*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***")
aug_orig_test_report = evaluate_model_metrics(aug_model, orig_test_generator)
aug_aug_test_report = evaluate_model_metrics(aug_model, aug_test_generator)

*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***
1792/1792 [==============================] - 354s 198ms/step - loss: 0.2004 - accuracy: 0.9297
Test Accuracy: 0.9296678900718689
21580/21580 [==============================] - 1377s 64ms/step - loss: 0.2150 - accuracy: 0.9257
Test Accuracy: 0.9257182478904724
21580/21580 [==============================] - 1380s 64ms/step


In [24]:
print("*** EVALUATIONS ON ORIGINAL TEST DATA WITH MODEL TRAINED ON AUGMENTED DATA ***")
aug_orig_df = pd.DataFrame(aug_orig_test_report).transpose()

print(f"Accuracy: {aug_orig_df.loc['accuracy', 'precision']}")
print(f"Macro Average Precision: {aug_orig_df.loc['macro avg', 'precision']}")
print(f"Macro Average Recall: {aug_orig_df.loc['macro avg', 'recall']}")
print(f"Macro Average F1-Score: {aug_orig_df.loc['macro avg', 'f1-score']}")

display(aug_orig_df)

print("*** EVALUATIONS ON AUGMENTED TEST DATA WITH MODEL TRAINED ON AUGMENTED DATA ***")
aug_aug_df = pd.DataFrame(aug_aug_test_report).transpose()

print(f"Accuracy: {aug_aug_df.loc['accuracy', 'precision']}")
print(f"Macro Average Precision: {aug_aug_df.loc['macro avg', 'precision']}")
print(f"Macro Average Recall: {aug_aug_df.loc['macro avg', 'recall']}")
print(f"Macro Average F1-Score: {aug_aug_df.loc['macro avg', 'f1-score']}")

display(aug_aug_df)


*** EVALUATIONS ON ORIGINAL TEST DATA WITH MODEL TRAINED ON AUGMENTED DATA ***
Accuracy: 0.9296678760814959
Macro Average Precision: 0.9392576863536143
Macro Average Recall: 0.9319632752592761
Macro Average F1-Score: 0.9305437187308214


,precision,recall,f1-score,support
Apple___Apple_scab,0.876866,0.991561,0.930693,237.000000
Apple___Black_rot,1.000000,0.925094,0.961089,267.000000
Apple___Cedar_apple_rust,1.000000,0.883621,0.938215,232.000000
Apple___healthy,0.958175,0.988235,0.972973,255.000000
Cherry_(including_sour)___Powdery_mildew,0.994792,0.994792,0.994792,192.000000
Cherry_(including_sour)___healthy,0.990741,0.995349,0.993039,215.000000
Corn_(maize)___Common_rust_,1.000000,0.963303,0.981308,218.000000
Corn_(maize)___Northern_Leaf_Blight,0.947826,1.000000,0.973214,218.000000
Corn_(maize)___healthy,0.995984,0.992000,0.993988,250.000000
Grape___Black_rot,0.995238,0.842742,0.912664,248.000000


*** EVALUATIONS ON AUGMENTED TEST DATA WITH MODEL TRAINED ON AUGMENTED DATA ***
Accuracy: 0.9257182576459685
Macro Average Precision: 0.9361833739752538
Macro Average Recall: 0.926610081205684
Macro Average F1-Score: 0.9268874239321236


,precision,recall,f1-score,support
Apple___Apple_scab,0.906716,0.964286,0.934615,756.000000
Apple___Black_rot,1.000000,0.912752,0.954386,745.000000
Apple___Cedar_apple_rust,1.000000,0.850000,0.918919,660.000000
Apple___healthy,0.944872,0.978752,0.961513,753.000000
Cherry_(including_sour)___Powdery_mildew,0.976599,0.992076,0.984277,631.000000
Cherry_(including_sour)___healthy,0.988270,0.983942,0.986101,685.000000
Corn_(maize)___Common_rust_,1.000000,0.953846,0.976378,715.000000
Corn_(maize)___Northern_Leaf_Blight,0.935864,1.000000,0.966870,715.000000
Corn_(maize)___healthy,0.997114,0.991392,0.994245,697.000000
Grape___Black_rot,0.989147,0.901130,0.943089,708.000000


In [27]:
create_model_confusion_matrix(orig_model, aug_test_generator, "orig model on aug test")

21580/21580 [==============================] - 1382s 64ms/step


NameError: name 'test_df' is not defined